#                      -- DESASTER --
Simulating household reconstruction with Discrete Event Simulation 

Requires python 3.4

In [1]:
#stdlib and 3rd Party imports
import sys, datetime, os
import numpy as np
import pandas as pd
import simpy
from simpy.util import start_delayed

#add path to desaster module, later we'll install this into site-packages so we shouldn't need to do this
sys.path.append("C:\\Users\\Derek\\desktop\\desaster")
#import desaster files
from desaster import entities, capitals, request, io, movement, search, rebuild


Here we're importing the modules and setting up the stuff

## Load input files for the scenario

In [2]:
#scenario_file = '../inputs/scenario_test1.xlsx'
scenario_file = "../inputs/household_inputs.xlsx"

# Create Pandas dataframe of attribute data for all households to be modeled in the simulation
# required column names, exactly as written: Name , Savings , Insurance
households_df = pd.read_excel(scenario_file, sheetname='households')

# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
housing_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

### Randomize households and reset index

In [3]:
households_df = households_df.sample(frac=1).reset_index(drop=True)
#takes a random sample, frac is a fraction to sample (1 means take a 100% sample), 
#reset index drops the old scrambled index and puts in a fresh ascending count
households_df.head()

,ID Number,Name,Value,Area,Content_Value,Damage State,Latitude,Longitude,Parcel_ID,Occupancy,Has Insurance,Savings,Insurance
0,US002135,JULIANE DOCKERY,21300.0,672.00,10650.0,Extensive,46.461534,-124.038344,74073015003,Mobile Home,0,2345.310887,0
1,US003332,AGNES WILLIS,135200.0,3008.00,67600.0,Slight,46.486053,-124.055537,12113323187,Single Family Dwelling,1,4593.936227,135200
2,US002188,SHIRA CEJA,27200.0,568.00,13600.0,Slight,46.447980,-124.054945,11110933114,Single Family Dwelling,0,2560.732151,0
3,US002578,TIFANY AIELLO,8000.0,450.00,4000.0,Extensive,46.361449,-124.052742,10110933179,Mobile Home,0,1552.973746,0
4,US001879,MENDY JAIMES,19100.0,674.95,9550.0,Slight,46.467939,-124.051122,74027000004,Single Family Dwelling,0,2219.293668,0


# Initiate Simulation

In [4]:
simulation = simpy.Environment() 

In [5]:
write_story = True #do we want the story of each household?


In [6]:
human_cap_data

Type
Contractors             40
Engineers                1
FEMA Processors         40
Inspectors             100
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [7]:
human_cap_data["Contractors"] = 100
human_cap_data["Engineers"] = 100
human_cap_data["FEMA Processors"]= 100
human_cap_data["Inspectors"] = 100
human_cap_data["Insurance Adjusters"] = 100
human_cap_data["Loan Processors"] = 100
human_cap_data["Permit Processors"] = 100


In [8]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) #resource

human_capital = capitals.HumanCapital(simulation, human_cap_data) #resource

households = entities.importHouseholds(simulation, households_df, write_story) #entity object container

housing_stock = capitals.importHousingStock(simulation, housing_stock_df) #available housing

### Write a function that controls the flow for each household.

In [9]:
def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 1000  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))

In [10]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], write_story))

In [11]:
undamaged_housing = 0
for i in housing_stock.items:
    if i.damage_state == "None":
        undamaged_housing += 1
        
print (undamaged_housing)

16


## Rebuild the housing stock

In [12]:
# Do inspections on all of the vacant homes in the housing stock
for home in housing_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

<Process(starter) object at 0x8d0c590>

# Parameters for the run

In [13]:
human_cap_data

Type
Contractors            100
Engineers              100
FEMA Processors        100
Inspectors             100
Insurance Adjusters    100
Loan Processors        100
Permit Processors      100
Name: Quantity, dtype: int64

## Run the model

In [14]:
#Reload building material at a preordained time
start_delayed(simulation, capitals.reloadBuildingMaterial(simulation, financial_capital.building_materials, amount = 100000000), 100)

simulation.run()

313 homes in the vacant building stock were fixed on day 100.


## Outputs to verify model ran correctly

In [16]:
num_undamaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_gave_up_home_search = 0
num_moved = 0
num_loaned = 0
for household in households:
    if household.money_search_start == None: num_undamaged += 1
    if household.home_get is not None: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1   
    if household.home_search_stop is not None:
        if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
    if household.entered_loan_process: num_loaned += 1    
print('{0} out of {1} households suffered no damage to their homes.\n'.format(num_undamaged, len(households)),
      '{0} out of {1} households rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(households)),
        '{0} out of {1} households gave up searching for money.\n'.format(num_gave_up_money_search, len(households)),
        '{0} out of {1} households decided to find a new home.\n'.format(num_relocated, len(households)),
        '{0} out of {1} households gave up searching for a home. \n'.format(num_gave_up_home_search, len(households)),
      '{0} out of {1} households loaned'.format(num_loaned, len(households))
     )

46 out of 2860 households suffered no damage to their homes.
 2759 out of 2860 households rebuilt or repaired their damaged home.
 0 out of 2860 households gave up searching for money.
 58 out of 2860 households decided to find a new home.
 1 out of 2860 households gave up searching for a home. 
 129 out of 2860 households loaned


In [ ]:
min(30,5)

# MAKE A NEW DATAFRAME FOR EXPORT

In [ ]:
#fills the empty dataframe we made above for the output. incredibly badly written
a = list(vars(households[4]).keys()) #gets all potential column names
a.remove("household");a.remove("residence") #remove the stuff we don't want
a.append("latitude");a.append("longitude") #add stuff we do want
df = pd.DataFrame(columns=a)
iters = 0
att_itter = 0
new_column={}
log = []
for i in households: #loop through all entities
    i.latitude = i.household["Latitude"] #extracting lat and long from the residence object
    i.longitude = i.household["Longitude"]
    for att in a: #loop through the attributes in our list of column names we want
        try:
            new_column[att] = i.__getattribute__(att) #set the b dictionary
            #mydata[att]= i.__getattribute__(att)
            
        except ValueError:
            new_column[att] = np.nan
        except AttributeError as e:
            new_column[att] = np.nan
            log.append("Household {0} had an attr error, {1}".format(i.name, e))
        finally:
            att_itter += 1
    mydata=pd.DataFrame([new_column]) #this turns our newly made column into a database where it can be combined with the df

    df = df.append(mydata, ignore_index=True)

    iters += 1
    

print(iters)
print(att_itter)


In [ ]:
df.head()
output_path = "../outputs/Output{}.csv".format(str(datetime.date.today()))
df.to_csv(output_path)

In [ ]:
#TODO Go in and change the "story" variable to format the strings nice. change it in the root code so its not janky
#and lines up correctly.

In [ ]:
df["wait"]=df["inspection_get"]-df["inspection_put"]

In [ ]:
df.entered_loan_process

In [ ]:
df.columns


In [ ]:
df.assessment_get.isnull()